### DAT158: ML-assignment-2-Project-work

In [38]:
import pandas as pd

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")


In [41]:
df_test.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [16]:
df_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [43]:
df_train = df_train.drop(columns=["id"])
df_test = df_test.drop(columns=["id"])

In [44]:
df_train.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [45]:
df_test.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        3383
engine              0
transmission        0
ext_col             0
int_col             0
accident         1632
clean_title     14239
dtype: int64

In [46]:
df_train.columns

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')

In [47]:
df_test.columns

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title'],
      dtype='object')

In [48]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 17.3+ MB


In [24]:
import seaborn as sns

missing_train = df_train.isna().mean() * 100
missing_test = df_test.isna().mean() * 100

print("Kolonner med mer enn 10% manglende verdier i df_train:")
print(missing_train[missing_train > 0])

print("\nKolonner med mer enn 10% manglende verdier i df_test:")
print(missing_test[missing_test > 0])

Kolonner med mer enn 10% manglende verdier i df_train:
fuel_type       2.696080
accident        1.300568
clean_title    11.360876
dtype: float64

Kolonner med mer enn 10% manglende verdier i df_test:
fuel_type       2.691543
accident        1.298433
clean_title    11.328666
dtype: float64


In [ ]:
# Analyserer numeriske og kategoriske kolonner separat
numerical_columns = df_train.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df_train.select_dtypes(include=['object']).columns

print("Numeriske kolonner:", numerical_columns.tolist())
print("\nKategoriske kolonner:", categorical_columns.tolist())

Numerical columns: ['model_year', 'milage', 'price']

Categorical columns: ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']


In [ ]:

#For numeriske kolonner fyller vi manglende verdier med medianen
for col in numerical_columns:
    if df_train[col].isnull().any():
        median_value = df_train[col].median()
        df_train[col].fillna(median_value, inplace=True)
        df_test[col].fillna(median_value, inplace=True)
        print(f"Fylt inn mangelnde verdier i {col} med median: {median_value}")

#For kategoriske kolonner fyller vi manglende verdier med den mest forekommende verdien
for col in categorical_columns:
    if df_train[col].isnull().any():
        mode_value = df_train[col].mode()[0]
        df_train[col].fillna(mode_value, inplace=True)
        df_test[col].fillna(mode_value, inplace=True)
        print(f"Fylt inn mangelnde verdier i {col} med modus: {mode_value}")

# Dobbeltsjekk for å sikre at alle manglende verdier er håndtert
print("\nManglende verdier etter rydding:")
print("Training set:\n", df_train.isnull().sum().sum())
print("Test set:\n", df_test.isnull().sum().sum())

Filled missing values in fuel_type with mode: Gasoline
Filled missing values in accident with mode: None reported
Filled missing values in clean_title with mode: Yes

Missing values after cleaning:
Training set:
 0
Test set:
 0
